In [1]:
import numpy as np
import pandas as pd 

In [2]:
df=pd.read_csv("data.csv")

In [3]:
df.head()

,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,Heart Disease
0,70,1,4,130,322,0,2,109,0,2.4,2,3,3,Presence
1,67,0,3,115,564,0,2,160,0,1.6,2,0,7,Absence
2,57,1,2,124,261,0,0,141,0,0.3,1,0,7,Presence
3,64,1,4,128,263,0,0,105,1,0.2,2,1,7,Absence
4,74,0,2,120,269,0,2,121,1,0.2,1,1,3,Absence


In [4]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(df['Heart Disease'])
df['Heart Disease']=encoder.transform(df['Heart Disease'])

In [5]:
# Preparing X and Y variables
X = df.drop(columns=['Heart Disease'],axis=1)
X

,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium
0,70,1,4,130,322,0,2,109,0,2.4,2,3,3
1,67,0,3,115,564,0,2,160,0,1.6,2,0,7
2,57,1,2,124,261,0,0,141,0,0.3,1,0,7
3,64,1,4,128,263,0,0,105,1,0.2,2,1,7
4,74,0,2,120,269,0,2,121,1,0.2,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,52,1,3,172,199,1,0,162,0,0.5,1,0,7
266,44,1,2,120,263,0,0,173,0,0.0,1,0,7
267,56,0,2,140,294,0,2,153,0,1.3,2,0,3
268,57,1,4,140,192,0,0,148,0,0.4,2,0,6


In [6]:
y = df['Heart Disease']
y

0      1
1      0
2      1
3      0
4      0
      ..
265    0
266    0
267    0
268    0
269    1
Name: Heart Disease, Length: 270, dtype: int64

In [7]:
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler 
from sklearn.compose import ColumnTransformer
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [8]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())])

In [9]:
num_features = X.select_dtypes(exclude="object").columns

In [10]:
preprocessor=ColumnTransformer([
     ('num_pipeline',num_pipeline,num_features)])

In [11]:
X = preprocessor.fit_transform(X)

In [12]:
X.shape

(270, 13)

In [13]:

 preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 Index(['Age', 'Sex', 'Chest pain type', 'BP', 'Cholesterol', 'FBS over 120',
       'EKG results', 'Max HR', 'Exercise angina', 'ST depression',
       'Slope of ST', 'Number of vessels fluro', 'Thallium'],
      dtype='object'))])

In [14]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape, y_train.shape,y_test.shape

((216, 13), (54, 13), (216,), (54,))

In [15]:
# Convert the series to a DataFrame with a single column
y_test = y_test.to_frame()

# Access the underlying numpy array and reshape it
y_test = y_test.values.reshape(-1, 1)

In [16]:
y_test.shape

(54, 1)

In [17]:
# Convert the series to a DataFrame with a single column
y_train = y_train.to_frame()

# Access the underlying numpy array and reshape it
y_train = y_train.values.reshape(-1, 1)

In [18]:
def model_eval(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true,y_pred).ravel()
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    specificity = tn/(fp+tn)
    F1_score = 2*recall*precision/(recall+precision)
    
    result = {   "Accuracy": accuracy,
                 "Precision": precision,
                 "Recall": recall,
                 "Specificity": specificity,
                 "F1 Score": F1_score}
    return result , accuracy

In [19]:
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostClassifier


In [20]:
models = {
    "Logistic Regression": LogisticRegression(),
    "K-Neighbors Classifier": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest Classifier": RandomForestClassifier(),
    "CatBoosting Classifier": CatBoostClassifier(verbose=False),
    "AdaBoost Classifier":AdaBoostClassifier()
}
model_list = []
Accuracy=[]
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    # Evaluate Train and Test dataset
    Train,accuracy_train = model_eval(y_train, y_train_pred)

    Test,accuracy_test = model_eval(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print(Train)


    print('----------------------------------')
    
    print('Model performance for Test set')
    print(Test)

    Accuracy.append(accuracy_test)
    
    print('='*35)
    print('\n')

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/tmp/ipykernel_3303/2284520413.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train) # Train model


Logistic Regression
Model performance for Training set
{'Accuracy': 0.8379629629629629, 'Precision': 0.8333333333333334, 'Recall': 0.8080808080808081, 'Specificity': 0.8632478632478633, 'F1 Score': 0.8205128205128206}
----------------------------------
Model performance for Test set
{'Accuracy': 0.9074074074074074, 'Precision': 0.9, 'Recall': 0.8571428571428571, 'Specificity': 0.9393939393939394, 'F1 Score': 0.8780487804878048}


K-Neighbors Classifier
Model performance for Training set
{'Accuracy': 0.8564814814814815, 'Precision': 0.8541666666666666, 'Recall': 0.8282828282828283, 'Specificity': 0.8803418803418803, 'F1 Score': 0.8410256410256411}
----------------------------------
Model performance for Test set
{'Accuracy': 0.8148148148148148, 'Precision': 0.8235294117647058, 'Recall': 0.6666666666666666, 'Specificity': 0.9090909090909091, 'F1 Score': 0.7368421052631577}


Decision Tree
Model performance for Training set
{'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'Specificity':

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [21]:
pd.DataFrame(list(zip(model_list, Accuracy)), columns=['Model Name', 'Accuracy']).sort_values(by=["Accuracy"],ascending=False)

,Model Name,Accuracy
0,Logistic Regression,0.907407
4,CatBoosting Classifier,0.888889
5,AdaBoost Classifier,0.888889
3,Random Forest Classifier,0.833333
1,K-Neighbors Classifier,0.814815
2,Decision Tree,0.703704


In [22]:
# Thus Logistic Regression Gives a Accuracy of Around 90.7 Percent